In [ ]:
%matplotlib inline
import numpy as np
import subprocess
import matplotlib.pyplot as plt

In [ ]:
def get_SNR(file):
    with open(file, "rt")as f:
        a = f.readlines()
    lst = []
    for x in a:
        if "Value:" in x:
            lst.append(float(x.split("Value: ")[1]))
        #print(x.split("bla"))
    lst = lst[len(lst)//2 : ]
    return (np.array(lst).mean(), np.array(lst).std())

In [ ]:
def step1(file):
    with open(file, "rb")as f:
        _ = f.read()
    
    rcv = np.frombuffer(_, dtype = np.uint8)
    match = np.array([0, 0, 178, 62, 71, 168, 187, 253, 16, 38, 149, 55, 226, 170, 26], dtype = np.uint8)
    match_extended = np.tile(match, int(rcv.shape[0]/len(match)))
    
    idx = np.argmax([np.mean(np.equal(np.packbits(np.roll(np.unpackbits(rcv), i)), match_extended)) for i in range(15 * 8)])
    
    to_write = np.packbits(np.roll(np.unpackbits(rcv), idx))
    
    with open("output2.data", "wb") as f:
        f.write(to_write)

In [ ]:
def step2():
    process=subprocess.Popen(["python2", "BCH_Recoder.py"],
                             stdin=subprocess.PIPE,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)


    #time.sleep(10)

    # while os.stat("output.data").st_size < os.stat("orig_pinguim.bmp").st_size * 2:
    #     pass

    inputdata= b" "
    stdoutdata,stderrdata=process.communicate(input=inputdata)
    
    return

In [ ]:
def step3(file):
    with open(file, "rb")as f:
        _ = f.read()
    
    final = np.frombuffer(_, dtype = np.uint8)
    
    tx = np.array([0, 255, 40, 250, 37, 42, 186], dtype = np.uint8)
    match_extended_final = np.tile(tx, int(final.shape[0]/len(tx)))
    
    return (1 - np.mean(np.equal(np.unpackbits(final), np.unpackbits(match_extended_final))))

Gerar top_block.py no gnuradio, depois no terminal: python2 top_block.py -> output.txt

In [ ]:
%%time
step1("output.data")

In [ ]:
%%time
step2()

In [ ]:
%%time
step3("output3.data")

In [ ]:
get_SNR("output.txt"), step3("output3.data")

## <font color = red> Quanto mais baixo o SNR, maior a chance de ocorrer problemas na recepção, mais chance do resultado obtido ser ruim
    
# <font color = blue> Resultados obtidos simulados
  <font color = blue> SNR Média, SNR STD, BER

     
      

((6.431067191780822, 0.22504306409420496), 0.00010883928571425727) -> 2nd and 4th

((5.718607230095759, 1.1661024738189016), 0.0001303571428571404) -> Skew


In [ ]:
a = """
((11.87596506849315, 0.27409708998581855), 0.0)

((10.632518561643836, 0.5444092597158589), 0.0)

((9.643535136986301, 0.3791730134233486), 0.0)

((7.445212465753424, 0.3496959348609927), 7.767857142848555e-06)

((7.067593767123286, 0.2606505527249247), 3.607142857142698e-05)

((5.600923698630137, 0.3705099114197793), 0.0007822321428571088)

((3.955382602739726, 0.39780842285182433), 0.01158276785714285)

((3.628972671232876, 0.389460811463942), 0.01671410714285715)

((3.1698814383561644, 0.47569910301387486), 0.030747589285714305)

((6.431067191780822, 0.22504306409420496), 0.00010883928571425727)

((5.718607230095759, 1.1661024738189016), 0.0001303571428571404)

((6.4866798905608745, 0.3070860797276639), 0.00012205357142858198)

"""

In [ ]:
snr_avgs = []
snr_stds = []
bers = []

for line in a.splitlines():
    aux = line.replace("(", "").replace(")", "").split(",")
    if aux != [""]:
        snr_avgs.append(float(aux[0]))
        snr_stds.append(float(aux[1]))
        bers.append(float(aux[2]))

In [ ]:
import math

In [ ]:
10*math.log10(15/7)

In [ ]:
EbN0_min = 0
EbN0_max = 10
EbN0_range = range(EbN0_min, EbN0_max+1)

In [ ]:
simu = np.load("ber_simu.npy")

In [ ]:
theory = np.load("ber_theory.npy")

In [ ]:
theory

In [ ]:
simu

In [ ]:
np.array(snr_avgs) + 10*math.log10(15/7)

In [ ]:
plt.style.use('ggplot')
#plt.semilogy(EbN0_range, theory, "g-.", label = "BPSK + Diff.Encoding Teórico")
plt.semilogy(EbN0_range, simu, "b-o", label = "BPSK + BCH(15, 7) Simulado")
plt.semilogy(np.array(snr_avgs) + 10*math.log10(15/7), bers, "o", color = "brown", label = "BPSK + BCH(15, 7) LADAMS")
plt.legend()
plt.grid()

plt.title('Simulação BER - BPSK + Diff.Encoding')
plt.xlabel('Eb/N0 (dB)')
plt.ylabel('BER')
plt.savefig("Curva.png", dpi = 300)